In [1]:
import cv2
import numpy as np

# Load image
image = cv2.imread('../Dataset/data_testing_system/For_Testing_System/16.jpg')

normalized_image = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

# Crop image to 3:4 aspect ratio
height, width, channels = normalized_image.shape

target_ratio = 5 / 4

if height == width:
    # Gambar sudah persegi, tidak perlu cropping
    img = normalized_image
else:
    if height > width:
        crop_height = int(width * target_ratio)
        y = int((height - crop_height) / 2)
        x = 0
        cropped_image = normalized_image[y:y+crop_height, x:x+width]
        img = cv2.resize(cropped_image, (600, 800))
    else:
        crop_height = int(height * (1 / target_ratio))
        y = int((height - crop_height) / 2)
        x = 0
        cropped_image = normalized_image[y:y+crop_height, x:x+width]    
        img = cv2.resize(cropped_image, (600, 800))

# Konversi gambar ke skala abu-abu
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Thresholding pada gambar untuk memisahkan objek dari latar belakang
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

# Temukan kontur pada gambar yang telah dithreshold dan dapatkan kontur terbesar yang diasumsikan sebagai objek daun
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnt = max(contours, key=cv2.contourArea)

# Dapatkan koordinat bounding box (kotak pembatas) dari kontur
x, y, w, h = cv2.boundingRect(cnt)

# Buat mask dengan ukuran yang sama dengan gambar dan inisialisasi dengan nilai 0 (hitam)
mask = np.zeros(img.shape[:2], np.uint8)

# Tentukan model latar belakang (background) dan objek (foreground)
bgdModel = np.zeros((1, 65), np.float64)
fgdModel = np.zeros((1, 65), np.float64)

# Tentukan persegi panjang (rectangle) yang mencakup objek daun (ROI)
rect = (x, y, w, h)

# Algoritma GrabCut untuk menghapus latar belakang
cv2.grabCut(img, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)

# Buat mask dimana piksel non-nol menunjukkan objek (foreground) yang mungkin
mask2 = np.where((mask == cv2.GC_FGD) | (mask == cv2.GC_PR_FGD), 255, 0).astype('uint8')

# Balikkan mask ke negatif (putih menjadi hitam dan sebaliknya)
mask2_inv = cv2.bitwise_not(mask2)

# Buat hasil segmentasi dengan latar belakang putih
result = np.zeros_like(img)
result[np.where(mask2_inv == 255)] = (255, 255, 255)
result[np.where(mask2_inv == 0)] = img[np.where(mask2_inv == 0)]

roi = result[y:y+h, x:x+w]

cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

resized_roi = cv2.resize(roi, (1080, 1080))

show = cv2.resize(resized_roi, (512, 512))
# show3 = cv2.resize(normalized_image, (720, 1280))
cv2.imshow("hasil", show)
cv2.imshow("hasil2", img)
# cv2.imshow("hasil33", show3)
cv2.waitKey(0)
cv2.destroyAllWindows()